**I recommend you run the first code cell of this notebook immediately, to start provisioning drake on the cloud machine, then you can leave this window open as you [read the textbook](manipulation.csail.mit.edu/pick.html).**

# Notebook setup

The following cell will:
- on Colab (only), install Drake to `/opt/drake`, install Drake's prerequisites via `apt`, and add pydrake to `sys.path`.  This will take approximately two minutes on the first time it runs (to provision the machine), but should only need to reinstall once every 12 hours.  If you navigate between notebooks using Colab's "File->Open" menu, then you can avoid provisioning a separate machine for each notebook.
- launch a server for our 3D visualizer (MeshCat) that will be used for the remainder of this notebook.

You will need to rerun this cell if you restart the kernel, but it should be fast because the machine will already have drake installed.

In [ ]:
import importlib
import sys
from urllib.request import urlretrieve

# Install drake.
if 'google.colab' in sys.modules and importlib.util.find_spec('pydrake') is None:
  version='20200826'
  build='nightly'
  urlretrieve(f"https://drake-packages.csail.mit.edu/drake/{build}/drake-{version}/setup_drake_colab.py",
              "setup_drake_colab.py")
  from setup_drake_colab import setup_drake
  setup_drake(version=version, build=build)

# Install pyngrok.
server_args = []
if 'google.colab' in sys.modules:
  !pip install pyngrok
  server_args = ['--ngrok_http_tunnel']

# Start a single meshcat server instance to use for the remainder of this notebook.
from meshcat.servers.zmqserver import start_zmq_server_as_subprocess
proc, zmq_url, web_url = start_zmq_server_as_subprocess(server_args=server_args)


# TODO(russt): upstream this to drake
import numpy as np
from functools import partial

from IPython.display import display
from ipywidgets import FloatSlider, Layout

from pydrake.all import JointIndex

def joint_sliders_with_callbacks(
    robot, meshcat, root_context, extra_callback=None, 
    lower_limit=-10., upper_limit=10., resolution=0.01, length=200):
    """ will add a slider for each joint in plant, and set up a callback to set the context and call meshcat publish. """

    def _reshape(x, num):
        x = np.array(x)
        assert len(x.shape) <= 1
        return np.array(x) * np.ones(num)

    lower_limit = _reshape(lower_limit, robot.num_positions())
    upper_limit = _reshape(upper_limit, robot.num_positions())
    resolution = _reshape(resolution, robot.num_positions())

    meshcat.load()
    meshcat_context = meshcat.GetMyContextFromRoot(root_context)
    meshcat.Publish(meshcat_context)
    robot_context = robot.GetMyContextFromRoot(root_context)
    positions = robot.GetPositions(robot_context)
    def _slider_callback(change, index):
        positions[index] = change.new
        robot.SetPositions(robot_context, positions)
        meshcat.Publish(meshcat_context)
        if extra_callback:
            extra_callback(robot_context)

    k = 0
    for i in range(0, robot.num_joints()):
        joint = robot.get_joint(JointIndex(i))
        low = joint.position_lower_limits()
        upp = joint.position_upper_limits()
        for j in range(0, joint.num_positions()):
            index = joint.position_start() + j
            slider = FloatSlider(
                value=positions[index],
                min=max(low[j], lower_limit[k]),
                max=min(upp[j], upper_limit[k]),
                step=resolution[k],
                continuous_update=True,
                description=joint.name(),
                style={'description_width': 'initial'},
                layout=Layout(width=f"'{length}'"))
            slider.observe(partial(_slider_callback, index=index), names='value')
            display(slider)
            k += 1


# Inspecting the kinematic tree

Here is a simple example that demonstrates how to inspect the kinematic tree stored in a `MultibodyPlant`.

In [ ]:
from pydrake.all import FindResourceOrThrow, MultibodyPlant, Parser, plot_graphviz

plant = MultibodyPlant(time_step=0.0)
parser = Parser(plant)
parser.AddModelFromFile(FindResourceOrThrow(
    "drake/manipulation/models/allegro_hand_description/sdf/allegro_hand_description_right.sdf"))
parser.AddModelFromFile(FindResourceOrThrow(
    "drake/examples/manipulation_station/models/061_foam_brick.sdf"))
plant.Finalize()

# TODO(russt): Update this to a javascript rendering.
# TODO(russt): Add floating base connections.
plot_graphviz(plant.GetTopologyGraphvizString());

# Forward kinematics of the gripper frame

Here is a simple example that let's you visualize the frames on the iiwa and the gripper.  If you click on the "Open Controls" menu in the MeshCat visualizer, and dig into the menu `meshcat->drake->Source` then you will see elements for each of the models in the `SceneGraph`: one for the iiwa, another for the WSG, and others for the clutter bins.  You can enable/disable their visualization.  Give it a spin!

In [ ]:
from IPython.display import display
from ipywidgets import Text, Layout

from pydrake.all import DiagramBuilder, ConnectMeshcatVisualizer, RollPitchYaw
from pydrake.examples.manipulation_station import ManipulationStation

builder = DiagramBuilder()

# TODO: Replace this with a simple model directive of iiwa+wsg (no clutter bins)
station = builder.AddSystem(ManipulationStation())
station.SetupClutterClearingStation()
station.Finalize()

frames_to_draw = {"iiwa": {"iiwa_link_1", "iiwa_link_2", "iiwa_link_3", "iiwa_link_4", "iiwa_link_5", "iiwa_link_6", "iiwa_link_7"},
                  "gripper": {"body"}}
meshcat = ConnectMeshcatVisualizer(builder,
    station.get_scene_graph(),
    output_port=station.GetOutputPort("pose_bundle"),
    zmq_url=zmq_url,
    frames_to_draw=frames_to_draw,
    axis_length=0.3,
    axis_radius=0.01)

diagram = builder.Build()
context = diagram.CreateDefaultContext()

xyz = Text(value="", description="gripper position (m): ", layout=Layout(width='500px'), style={'description_width':'initial'})
rpy = Text(value="", description="gripper roll-pitch-yaw (rad): ", layout=Layout(width='500px'), style={'description_width':'initial'})
plant = station.get_multibody_plant()

gripper = plant.GetBodyByName("body")
def pose_callback(context):
    pose = plant.EvalBodyPoseInWorld(context, gripper)   ## This is the important line
    xyz.value = np.array2string(pose.translation(), formatter={'float': lambda x: "{:3.2f}".format(x)})
    rpy.value = np.array2string(RollPitchYaw(pose.rotation()).vector(), formatter={'float': lambda x: "{:3.2f}".format(x)})

joint_sliders_with_callbacks(station.get_multibody_plant(), meshcat, context, extra_callback=pose_callback)
display(xyz)
display(rpy)


# Don't assume $\dot{q} \equiv v$

# Kinematic Jacobians for pick and place

Let's set up the same iiwa + wsg example, with sliders (but without the frames), that we used above.  But this time I'll display the value of the Jacobian $J^G(q)$.

In [ ]:
from IPython.display import display
from ipywidgets import Text, Textarea, Layout

from pydrake.all import (DiagramBuilder, ConnectMeshcatVisualizer, JacobianWrtVariable)
from pydrake.examples.manipulation_station import ManipulationStation

builder = DiagramBuilder()

# TODO: Replace this with a simple model directive of iiwa+wsg (no clutter bins)
station = builder.AddSystem(ManipulationStation())
station.SetupClutterClearingStation()
station.Finalize()

meshcat = ConnectMeshcatVisualizer(builder,
    station.get_scene_graph(),
    output_port=station.GetOutputPort("pose_bundle"),
    zmq_url=zmq_url)

diagram = builder.Build()
context = diagram.CreateDefaultContext()

jacobian = Textarea(value="", description="J_G: ", layout={'width':'60%','height':'100px'}, style={'description_width':'initial'})
sigma = Text(value="", description="smallest singular value(J_G): ", layout={'width':'60%'}, style={'description_width':'initial'})
plant = station.get_multibody_plant()

G = plant.GetBodyByName("body").body_frame()
W = plant.world_frame()
def pose_callback(context):
    J_G = plant.CalcJacobianSpatialVelocity(context, JacobianWrtVariable.kQDot, G, [0,0,0], W, W)   ## This is the important line
    jacobian.value = np.array2string(J_G, formatter={'float': lambda x: "{:5.1f}".format(x)})
    sigma.value = str(np.min(np.linalg.svd(J_G, compute_uv=False)))

joint_sliders_with_callbacks(station.get_multibody_plant(), meshcat, context, extra_callback=pose_callback)
display(jacobian)
display(sigma)
